In [73]:
!pip install OWSLib

^C
Operation cancelled by user


## Testing API access to S-122 data through an OGC API.

In [82]:
# something on OGC API
from owslib.ogcapi.features import Features


## Access to OGC API via OWS lib interfaces. 
List out all the collections...

In [83]:
w = Features('http://18.130.213.16/ogcfmsdi/')
feature_collections = w.feature_collections()
print(feature_collections)

['S-100Vector', 'S-122WDPA', 'S-122Dev', 'S-122JNCC', 'S-122DNK', 'S-122DEU', 'TEST']


In [84]:
denmark = w.collection_items('S-122DNK')

## Complex attributes
they're not strings, they're JSON dictionaries, you can filter on these though, but you have to escape all the 's and get the formatting right, it's not ideal but it is possible to make up a search string on just the name (if you know the language). What I'm not sure is whether you can do a more general search. pygeoapi only supports simple profile of CQL though which hampers things somewhat.

In [85]:
print(denmark['features'][0]['properties']['featureName']['name'])

Nysø, Bøndernes Egehoved


In [86]:
print(denmark['features'][3]['properties']['featureName']['name'])
print(denmark['features'][3]['properties']['featureName'])
d4 = w.collection_items('S-122DNK', featureName='{\'name\': \'Ertholmene\', \'language\': \'DNK\'}')
print(d4['numberMatched'])


Ertholmene
{'name': 'Ertholmene', 'language': 'DNK'}
3


## How to retrieve features by relationship

Each relationship can be included as a distinct link (although we may have to group these by relationship name as there is a multiplicity of relationships between features and information types). so, it is possible to grab features, examine the inter-feature links and retrieve attribution from those links. This example retrieves the associated Authority feature and looks for the categoryOfAuthority it contains.

In [87]:
denmark2 = w.collection_items('TEST')
print(denmark2['features'][0]['id'])
print(denmark2['features'][0]['links'])
print( (denmark2['features'][0]['links']).keys() )
relatedlink = denmark2['features'][0]['links']['protectedAreaAuthority']
print('->' + relatedlink)
d3 = w.collection_item('TEST',str(relatedlink))
print(d3['properties']['categoryOfAuthority'])


ID0
{'protectedAreaAuthority': 'ID1'}
dict_keys(['protectedAreaAuthority'])
->ID1
environmental


In [88]:
jncc1 = w.collection_items('S-122JNCC')
print(jncc1['features'][1])

relatedlink = jncc1['features'][1]['links']['protectedAreaAuthority']
print('->' + relatedlink)
d3 = w.collection_item('S-122JNCC',str(relatedlink))
print(d3['properties']['featureName']['name'])

{'type': 'Feature', 'id': 'UKNCMPA020', 'properties': {'featureType': 'MarineProtectedArea', 'featureName': {'name': 'Central Fladen'}, 'categoryOfMarineProtectedArea': 'Unknown', 'status': 'permanent'}, 'geometry': {'type': 'MultiPolygon', 'coordinates': [[[[0.108983607507525, 58.8551780583208], [-0.002838612811672, 58.8190508779957], [-0.118068032513634, 58.8271201781537], [-0.164053432719697, 58.8303137184034], [-0.405296772128281, 58.9595311581365], [-0.526745052342221, 59.0931489977281], [-0.377357232849988, 59.3179229176086], [-0.207963493520367, 59.2551523980648], [-0.100887752867436, 59.1926915375844], [-0.288270292939615, 59.0618211781049], [-0.13955399272254, 58.9874630378889], [0.074576767303995, 58.970440977769], [0.096941226656327, 58.9239917378285], [0.108983607507525, 58.8551780583208]]]]}, 'links': {'protectedAreaAuthority': 'JNCC'}}
->JNCC
JNCC


## Filtering.
This retrieves features according to the IUCN category. Note that the category is not conformant with the S-122 enumeration values (yet). It would be possible to read in a JSON version of the feature catalogue and establish what these are (next step in this notebook).

In [89]:
denmark4 = w.collection_items('S-122DNK', categoryOfMarineProtectedArea='IV')
print(denmark4['numberMatched'])
print(denmark4['features'][1]['properties']['featureName'])

germany4 = w.collection_items('S-122DEU', categoryOfMarineProtectedArea='IV')
print(germany4['numberMatched'])
print(germany4['features'][1]['properties']['featureName'])

26
{'name': 'Ulvshale', 'language': 'DNK'}
66
{'name': 'Insel Vilm', 'language': 'DEU'}


## other ways?

In [90]:
import requests
url='http://18.130.213.16/ogcfmsdi/collections/S-122JNCC/items/JNCC?f=json&'
url='http://18.130.213.16/ogcfmsdi/collections/S-122JNCC/items?f=json&limit=1000'
url='http://18.132.248.81:5000/collections/obs/items?f=json'
callurl = url + '&filter-lang=cql&filter=categoryOfMarineProtectedArea LIKE %V%'
response = requests.get(callurl)
print()
resp = response.json()
print(response.json())


{'code': 'InvalidParameterValue', 'description': 'unknown query parameter: filter-lang'}
